El teorema de Bayes
P(A|B) = P(A) P(A|B) / P(B)

* ¿Como usarlo con machine learning para construir un clasificador?
Ejemplo. Expresar la probabilidad de que un email sea spam si contiene 
la palabra "Free".

* P(SPAM|Free) = P(SPAM) P(Free|SPAM) / P(Free)

* Donde el numerador represente la probabilidad de que el mensaje sea SPAM
   contenga la palabra "Free".

* Y el denominador es la probabilidad de que el mensaje contenga la palabra "Free"
 
* El modelo nos dara un conjunto en % de emails que contengan la palabra "Free" 
  que son SPAM
  
 Suena complicado y mucho trabajo pero
 
 * Scikit-learn al rescate!!  
 
 * Esta biblioteca contiene un metodo llamado CountVectorizer que nos permite
   trabajar con grandes cantidades de palabras, tambien contienen el metodo
   MultinomialNB que el trtabajo pesado de Naive Bayes.
   
 * Entrenaremos un conjunto donde se conoce que tienen Spam y "HAM" donde
   "HAM" son los no SPAM, esto conoce como Aprendizaje Supervisado.
   
A codificar!!  

In [105]:
import  sklearn

In [106]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [107]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root,filename)
            
            inBody = False
            lines =[]
            f = io.open(path,'r', encoding='latin1')
            for line in f:
              if inBody:
                    lines.append(line)
              elif line == '\n' :
                  inBody=True
            f.close()
            message = '\n'.join(lines)
            yield path, message
                    
        
        

In [108]:
def dataFrameFromDirectory(path, classification):
    rows =[]
    index =[]
    for filename, message in readFiles(path):
        rows.append({'message': message,'class':classification})
        index.append(filename)
        
    return DataFrame(rows, index=index)    


In [122]:
data = DataFrame({'message':[],'class':[]})

data = data.append(dataFrameFromDirectory('emails/spam','spam'))
data = data.append(dataFrameFromDirectory('emails/ham','ham'))


In [110]:
data.head

<bound method DataFrame.head of                                                    class  \
emails/spam/00334.a1038f98abb76b403d068afb57bfb290  spam   
emails/spam/00446.a54877313142d56c24d499d761c48fb1  spam   
emails/spam/00419.141092086514a246ff2ff8d4bc523400  spam   
emails/spam/00467.5b733c506b7165424a0d4a298e67970f  spam   
emails/spam/00063.2334fb4e465fc61e8406c75918ff72ed  spam   
emails/spam/00176.79f82496c612ea28f45f13ca5c47f8c2  spam   
emails/spam/00250.32279787338af8a5de4cfbc0b837718e  spam   
emails/spam/00298.90b548a0816ca0783f012bb9c69166cc  spam   
emails/spam/00297.3350c2dbbb0272c27b2c7773d7012356  spam   
emails/spam/00421.ca2fe949a956845a9ba81c649a7db6c0  spam   
emails/spam/00112.be81f2f6f7940a9403c9809b4a9e243a  spam   
emails/spam/00146.e9b64856c0cd982a64f47c9ab9084287  spam   
emails/spam/00465.ca5d79d0e5dadee322c117789196ebb4  spam   
emails/spam/00084.a9f5b3a9b7feb7070f25ae76320c8ec6  spam   
emails/spam/00439.6f4246a5e3336b6ecb5624e209e0b59f  spam   
emails/s

In [111]:
vectorizer=CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values

classifier.fit(counts, targets)





MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [112]:
examples = [' Viagra now  free !!!', 'Hi, Free  Free  free Bob, how about a game of golf tomorrow']
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], 
      dtype='|S4')

In [123]:






# train = 2500
# test = 500
# Aplicar un slidezing
import pandas as pd

# Funcion de corte junto con entrenamientos
def checarFun(train, test):
    #
    vectorizer = CountVectorizer()
    counts = vectorizer.fit_transform(data['message'][:train].values) # partir a 2500
    # training
    classifier = MultinomialNB()
    targets = data['class'][:train].values # esta bien a 2500
    classifier.fit(counts, targets)
    
    # prediction
    examples = data['message'][test:]
    example_counts = vectorizer.transform(examples)
    predictions = classifier.predict(example_counts)
    #
    return pd.Series(predictions).value_counts()

#















In [124]:
checarFun(2500,500)

ham     2494
spam       6
dtype: int64

In [125]:
checarFun(2000,1000)


ham     1995
spam       5
dtype: int64